## Waste Classification Model
This project aims to classify images of waste materials into recyclable and non-recyclable categories.

In [ ]:

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import cv2
from tqdm import tqdm
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings("ignore")


In [ ]:

# Define paths to training and test directories
train_directory = "../input/non-and-biodegradable-waste-dataset/TRAIN.1"
test_directory = "../input/non-and-biodegradable-waste-dataset/TEST"
img_size = (60, 60)


In [ ]:

# Function to load images and labels
def load_data(directory, img_size):
    images, labels = [], []
    for label_name in ('B', 'N'):
        label = 0 if label_name == 'B' else 1  # 0 for biodegradable, 1 for non-biodegradable
        path = os.path.join(directory, label_name)
        for img_name in tqdm(os.listdir(path), desc=f"Loading {label_name}"):
            img_path = os.path.join(path, img_name)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, img_size) / 255.0  # normalize
                images.append(img)
                labels.append(label)
    return np.array(images, dtype='float32'), np.array(labels, dtype='int32')

# Load train and test data
train_images, train_labels = load_data(train_directory, img_size)
test_images, test_labels = load_data(test_directory, img_size)

# Print dataset shapes
print(f"Train data shape: {train_images.shape}, Train labels shape: {train_labels.shape}")
print(f"Test data shape: {test_images.shape}, Test labels shape: {test_labels.shape}")


In [ ]:

# Visualize the class distribution
sns.countplot(x=train_labels)
plt.title("Class Distribution")
plt.xlabel("Class Names (0: Biodegradable, 1: Non-Biodegradable)")
plt.ylabel("Count")
plt.show()


In [ ]:

# Initialize the VGG16 model for binary classification
vgg16_base = VGG16(include_top=False, weights="imagenet", input_shape=(60, 60, 3), pooling='max')
for layer in vgg16_base.layers:
    layer.trainable = False  # Freeze VGG16 base layers

# Create a sequential model and add layers
model = Sequential([
    vgg16_base,
    Flatten(),
    Dense(1, activation='sigmoid')  # Binary classification: recyclable vs non-recyclable
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(train_images, train_labels, validation_split=0.2, epochs=20, batch_size=32, callbacks=[early_stopping])


In [ ]:

# Model evaluation on test data
test_loss, test_accuracy = model.evaluate(test_images, test_labels, verbose=0)
print(f"Test accuracy: {test_accuracy:.2f}")

# Predictions
predictions = (model.predict(test_images) > 0.5).astype("int32")


In [ ]:

# Generate confusion matrix and classification report
cm = confusion_matrix(test_labels, predictions)
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=["Biodegradable", "Non-Biodegradable"], yticklabels=["Biodegradable", "Non-Biodegradable"])
plt.xlabel("Predicted")
plt.ylabel("Actual")
plt.title("Confusion Matrix")
plt.show()

# Print classification report
print("Classification Report:")
print(classification_report(test_labels, predictions, target_names=["Biodegradable", "Non-Biodegradable"]))
